In [1]:
import numpy as np
import pandas as pd

In [2]:
onsides_adverse_reactions = pd.read_csv(
    '../onsides_20230309/adverse_reactions_active_labels.csv')[['ingredients_rxcuis', 'pt_meddra_term']]
onsides_adverse_reactions = onsides_adverse_reactions.groupby(
    'ingredients_rxcuis')['pt_meddra_term'].apply(set).apply(';'.join).reset_index()
onsides_adverse_reactions = onsides_adverse_reactions.rename(columns={'pt_meddra_term': 'adverse_reactions'})

In [3]:
onsides_boxed_warnings = pd.read_csv(
    '../onsides_20230309/boxed_warnings_active_labels.csv')[['ingredients_rxcuis', 'pt_meddra_term']]
onsides_boxed_warnings = onsides_boxed_warnings.groupby(
    'ingredients_rxcuis')['pt_meddra_term'].apply(set).apply(';'.join).reset_index()
onsides_boxed_warnings = onsides_boxed_warnings.rename(columns={'pt_meddra_term': 'boxed_warnings'})

In [4]:
input_df = pd.read_csv(
    '../data/conmed_example_data_with_best_rxcuid.csv',
    index_col=0).fillna(np.nan).replace([np.nan], [None])

In [5]:
input_df['Best RxNorm Id'] = [
    str(int(rxcui)) if rxcui else None for rxcui in input_df['Best RxNorm Id']]

In [6]:
onsides = onsides_adverse_reactions.merge(
    onsides_boxed_warnings,
    how='outer',
    on='ingredients_rxcuis')

In [7]:
output_df = input_df.merge(
    onsides,
    how='left',
    left_on='Best RxNorm Id',
    right_on='ingredients_rxcuis')

In [8]:
output_df.to_csv('../data/conmed_example_data_with_onsides.csv')